# Import necessary dependencies and settings

In [17]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt

pd.options.display.max_colwidth = 200
%matplotlib inline

# Sample corpus of text documents

In [2]:
corpus = ['The sky is blue and beautiful.',
          'Love this blue and beautiful sky!',
          'The quick brown fox jumps over the lazy dog.',
          "A king's breakfast has sausages, ham, bacon, eggs, toast and beans",
          'I love green eggs, ham, sausages and bacon!',
          'The brown fox is quick and the blue dog is lazy!',
          'The sky is very blue and the sky is very beautiful today',
          'The dog is lazy but the brown fox is quick!'    
]
labels = ['weather', 'weather', 'animals', 'food', 'food', 'animals', 'weather', 'animals']

corpus = np.array(corpus)
corpus_df = pd.DataFrame({'Document': corpus, 
                          'Category': labels})
corpus_df = corpus_df[['Document', 'Category']]
corpus_df

Document Category
0                                      The sky is blue and beautiful.  weather
1                                   Love this blue and beautiful sky!  weather
2                        The quick brown fox jumps over the lazy dog.  animals
3  A king's breakfast has sausages, ham, bacon, eggs, toast and beans     food
4                         I love green eggs, ham, sausages and bacon!     food
5                    The brown fox is quick and the blue dog is lazy!  animals
6            The sky is very blue and the sky is very beautiful today  weather
7                         The dog is lazy but the brown fox is quick!  animals

# Simple text pre-processing

In [3]:
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

In [4]:
norm_corpus = normalize_corpus(corpus)
norm_corpus

array(['sky blue beautiful', 'love blue beautiful sky',
       'quick brown fox jumps lazy dog',
       'kings breakfast sausages ham bacon eggs toast beans',
       'love green eggs ham sausages bacon',
       'brown fox quick blue dog lazy', 'sky blue sky beautiful today',
       'dog lazy brown fox quick'],
      dtype='<U51')

# Bag of Words Model

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
# get bag of words features in sparse format
cv = CountVectorizer(min_df=0., max_df=1.)
cv_matrix = cv.fit_transform(norm_corpus)
cv_matrix

<8x20 sparse matrix of type '<class 'numpy.int64'>'
	with 42 stored elements in Compressed Sparse Row format>

In [10]:
# view non-zero feature positions in the sparse matrix
print(cv_matrix)

  (0, 2)	1
  (0, 3)	1
  (0, 17)	1
  (1, 14)	1
  (1, 2)	1
  (1, 3)	1
  (1, 17)	1
  (2, 6)	1
  (2, 13)	1
  (2, 11)	1
  (2, 8)	1
  (2, 5)	1
  (2, 15)	1
  (3, 1)	1
  (3, 18)	1
  (3, 7)	1
  (3, 0)	1
  (3, 10)	1
  (3, 16)	1
  (3, 4)	1
  (3, 12)	1
  (4, 9)	1
  (4, 7)	1
  (4, 0)	1
  (4, 10)	1
  (4, 16)	1
  (4, 14)	1
  (5, 6)	1
  (5, 13)	1
  (5, 8)	1
  (5, 5)	1
  (5, 15)	1
  (5, 3)	1
  (6, 19)	1
  (6, 2)	1
  (6, 3)	1
  (6, 17)	2
  (7, 6)	1
  (7, 13)	1
  (7, 8)	1
  (7, 5)	1
  (7, 15)	1


In [11]:
# view dense representation 
# warning might give a memory error if data is too big
cv_matrix = cv_matrix.toarray()
cv_matrix

array([[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1],
       [0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]], dtype=int64)

In [7]:
# get all unique words in the corpus
vocab = cv.get_feature_names()
# show document feature vectors
pd.DataFrame(cv_matrix, columns=vocab)

bacon  beans  beautiful  blue  breakfast  brown  dog  eggs  fox  green  \
0      0      0          1     1          0      0    0     0    0      0   
1      0      0          1     1          0      0    0     0    0      0   
2      0      0          0     0          0      1    1     0    1      0   
3      1      1          0     0          1      0    0     1    0      0   
4      1      0          0     0          0      0    0     1    0      1   
5      0      0          0     1          0      1    1     0    1      0   
6      0      0          1     1          0      0    0     0    0      0   
7      0      0          0     0          0      1    1     0    1      0   

   ham  jumps  kings  lazy  love  quick  sausages  sky  toast  today  
0    0      0      0     0     0      0         0    1      0      0  
1    0      0      0     0     1      0         0    1      0      0  
2    0      1      0     1     0      1         0    0      0      0  
3    1      0      1     0     0      0         1    0      1      0  
4    1      0      0     0     1      0         1    0      0      0  
5    0      0      0     1     0      1         0    0      0      0  
6    0      0      0     0     0      0         0    2      0      1  
7    0      0      0     1     0      1         0    0      0      0

# Bag of N-Grams Model

In [8]:
# you can set the n-gram range to 1,2 to get unigrams as well as bigrams
bv = CountVectorizer(ngram_range=(2,2))
bv_matrix = bv.fit_transform(norm_corpus)

bv_matrix = bv_matrix.toarray()
vocab = bv.get_feature_names()
pd.DataFrame(bv_matrix, columns=vocab)

bacon eggs  beautiful sky  beautiful today  blue beautiful  blue dog  \
0           0              0                0               1         0   
1           0              1                0               1         0   
2           0              0                0               0         0   
3           1              0                0               0         0   
4           0              0                0               0         0   
5           0              0                0               0         1   
6           0              0                1               0         0   
7           0              0                0               0         0   

   blue sky  breakfast sausages  brown fox  dog lazy  eggs ham     ...       \
0         0                   0          0         0         0     ...        
1         0                   0          0         0         0     ...        
2         0                   0          1         0         0     ...        
3         0                   1          0         0         0     ...        
4         0                   0          0         0         1     ...        
5         0                   0          1         1         0     ...        
6         1                   0          0         0         0     ...        
7         0                   0          1         1         0     ...        

   lazy dog  love blue  love green  quick blue  quick brown  sausages bacon  \
0         0          0           0           0            0               0   
1         0          1           0           0            0               0   
2         1          0           0           0            1               0   
3         0          0           0           0            0               0   
4         0          0           1           0            0               1   
5         0          0           0           1            0               0   
6         0          0           0           0            0               0   
7         0          0           0           0            0               0   

   sausages ham  sky beautiful  sky blue  toast beans  
0             0              0         1            0  
1             0              0         0            0  
2             0              0         0            0  
3             1              0         0            1  
4             0              0         0            0  
5             0              0         0            0  
6             0              1         1            0  
7             0              0         0            0  

[8 rows x 29 columns]

# TF-IDF Model

In [21]:
from sklearn.feature_extraction.text import TfidfTransformer

tt = TfidfTransformer(norm='l2', use_idf=True, smooth_idf=True)
tt_matrix = tt.fit_transform(cv_matrix)

tt_matrix = tt_matrix.toarray()
vocab = cv.get_feature_names()
pd.DataFrame(np.round(tt_matrix, 2), columns=vocab)

bacon  beans  beautiful  blue  breakfast  brown   dog  eggs   fox  green  \
0   0.00   0.00       0.60  0.53       0.00   0.00  0.00  0.00  0.00   0.00   
1   0.00   0.00       0.49  0.43       0.00   0.00  0.00  0.00  0.00   0.00   
2   0.00   0.00       0.00  0.00       0.00   0.38  0.38  0.00  0.38   0.00   
3   0.32   0.38       0.00  0.00       0.38   0.00  0.00  0.32  0.00   0.00   
4   0.39   0.00       0.00  0.00       0.00   0.00  0.00  0.39  0.00   0.47   
5   0.00   0.00       0.00  0.37       0.00   0.42  0.42  0.00  0.42   0.00   
6   0.00   0.00       0.36  0.32       0.00   0.00  0.00  0.00  0.00   0.00   
7   0.00   0.00       0.00  0.00       0.00   0.45  0.45  0.00  0.45   0.00   

    ham  jumps  kings  lazy  love  quick  sausages   sky  toast  today  
0  0.00   0.00   0.00  0.00  0.00   0.00      0.00  0.60   0.00    0.0  
1  0.00   0.00   0.00  0.00  0.57   0.00      0.00  0.49   0.00    0.0  
2  0.00   0.53   0.00  0.38  0.00   0.38      0.00  0.00   0.00    0.0  
3  0.32   0.00   0.38  0.00  0.00   0.00      0.32  0.00   0.38    0.0  
4  0.39   0.00   0.00  0.00  0.39   0.00      0.39  0.00   0.00    0.0  
5  0.00   0.00   0.00  0.42  0.00   0.42      0.00  0.00   0.00    0.0  
6  0.00   0.00   0.00  0.00  0.00   0.00      0.00  0.72   0.00    0.5  
7  0.00   0.00   0.00  0.45  0.00   0.45      0.00  0.00   0.00    0.0

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(min_df=0., max_df=1., norm='l2',
                     use_idf=True, smooth_idf=True)
tv_matrix = tv.fit_transform(norm_corpus)
tv_matrix = tv_matrix.toarray()

vocab = tv.get_feature_names()
pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)

bacon  beans  beautiful  blue  breakfast  brown   dog  eggs   fox  green  \
0   0.00   0.00       0.60  0.53       0.00   0.00  0.00  0.00  0.00   0.00   
1   0.00   0.00       0.49  0.43       0.00   0.00  0.00  0.00  0.00   0.00   
2   0.00   0.00       0.00  0.00       0.00   0.38  0.38  0.00  0.38   0.00   
3   0.32   0.38       0.00  0.00       0.38   0.00  0.00  0.32  0.00   0.00   
4   0.39   0.00       0.00  0.00       0.00   0.00  0.00  0.39  0.00   0.47   
5   0.00   0.00       0.00  0.37       0.00   0.42  0.42  0.00  0.42   0.00   
6   0.00   0.00       0.36  0.32       0.00   0.00  0.00  0.00  0.00   0.00   
7   0.00   0.00       0.00  0.00       0.00   0.45  0.45  0.00  0.45   0.00   

    ham  jumps  kings  lazy  love  quick  sausages   sky  toast  today  
0  0.00   0.00   0.00  0.00  0.00   0.00      0.00  0.60   0.00    0.0  
1  0.00   0.00   0.00  0.00  0.57   0.00      0.00  0.49   0.00    0.0  
2  0.00   0.53   0.00  0.38  0.00   0.38      0.00  0.00   0.00    0.0  
3  0.32   0.00   0.38  0.00  0.00   0.00      0.32  0.00   0.38    0.0  
4  0.39   0.00   0.00  0.00  0.39   0.00      0.39  0.00   0.00    0.0  
5  0.00   0.00   0.00  0.42  0.00   0.42      0.00  0.00   0.00    0.0  
6  0.00   0.00   0.00  0.00  0.00   0.00      0.00  0.72   0.00    0.5  
7  0.00   0.00   0.00  0.45  0.00   0.45      0.00  0.00   0.00    0.0

# Understanding TF-IDF from Scratch

In [82]:
# get unique words as feature names
unique_words = list(set([word for doc in [doc.split() for doc in norm_corpus] 
                         for word in doc]))
def_feature_dict = {w: 0 for w in unique_words}
print('Feature Names:', unique_words)
print('Default Feature Dict:', def_feature_dict)

Feature Names: ['lazy', 'fox', 'love', 'jumps', 'sausages', 'blue', 'ham', 'beautiful', 'brown', 'kings', 'eggs', 'quick', 'bacon', 'breakfast', 'toast', 'beans', 'green', 'today', 'dog', 'sky']
Default Feature Dict: {'lazy': 0, 'fox': 0, 'kings': 0, 'love': 0, 'jumps': 0, 'sausages': 0, 'breakfast': 0, 'today': 0, 'brown': 0, 'ham': 0, 'beautiful': 0, 'green': 0, 'eggs': 0, 'blue': 0, 'bacon': 0, 'toast': 0, 'beans': 0, 'dog': 0, 'sky': 0, 'quick': 0}


In [83]:
from collections import Counter
# build bag of words features for each document - term frequencies
bow_features = []
for doc in norm_corpus:
    bow_feature_doc = Counter(doc.split())
    all_features = Counter(def_feature_dict)
    bow_feature_doc.update(all_features)
    bow_features.append(bow_feature_doc)

bow_features = pd.DataFrame(bow_features)
bow_features

bacon  beans  beautiful  blue  breakfast  brown  dog  eggs  fox  green  \
0      0      0          1     1          0      0    0     0    0      0   
1      0      0          1     1          0      0    0     0    0      0   
2      0      0          0     0          0      1    1     0    1      0   
3      1      1          0     0          1      0    0     1    0      0   
4      1      0          0     0          0      0    0     1    0      1   
5      0      0          0     1          0      1    1     0    1      0   
6      0      0          1     1          0      0    0     0    0      0   
7      0      0          0     0          0      1    1     0    1      0   

   ham  jumps  kings  lazy  love  quick  sausages  sky  toast  today  
0    0      0      0     0     0      0         0    1      0      0  
1    0      0      0     0     1      0         0    1      0      0  
2    0      1      0     1     0      1         0    0      0      0  
3    1      0      1     0     0      0         1    0      1      0  
4    1      0      0     0     1      0         1    0      0      0  
5    0      0      0     1     0      1         0    0      0      0  
6    0      0      0     0     0      0         0    2      0      1  
7    0      0      0     1     0      1         0    0      0      0

In [84]:
import scipy.sparse as sp
feature_names = list(bow_features.columns)

# build the document frequency matrix
df = np.diff(sp.csc_matrix(bow_features, copy=True).indptr)
df = 1 + df # adding 1 to smoothen idf later

# show smoothened document frequencies
pd.DataFrame([df], columns=feature_names)

bacon  beans  beautiful  blue  breakfast  brown  dog  eggs  fox  green  \
0      3      2          4     5          2      4    4     3    4      2   

   ham  jumps  kings  lazy  love  quick  sausages  sky  toast  today  
0    3      2      2     4     3      4         3    4      2      2

In [85]:
# compute inverse document frequencies
total_docs = 1 + len(norm_corpus)
idf = 1.0 + np.log(float(total_docs) / df)

# show smoothened idfs
pd.DataFrame([np.round(idf, 2)], columns=feature_names)

bacon  beans  beautiful  blue  breakfast  brown   dog  eggs   fox  green  \
0    2.1    2.5       1.81  1.59        2.5   1.81  1.81   2.1  1.81    2.5   

   ham  jumps  kings  lazy  love  quick  sausages   sky  toast  today  
0  2.1    2.5    2.5  1.81   2.1   1.81       2.1  1.81    2.5    2.5

In [87]:
# compute idf diagonal matrix  
total_features = bow_features.shape[1]
idf_diag = sp.spdiags(idf, diags=0, m=total_features, n=total_features)
idf_dense = idf_diag.todense()

# print the idf diagonal matrix
pd.DataFrame(np.round(idf_dense, 2))

0    1     2     3    4     5     6    7     8    9    10   11   12  \
0   2.1  0.0  0.00  0.00  0.0  0.00  0.00  0.0  0.00  0.0  0.0  0.0  0.0   
1   0.0  2.5  0.00  0.00  0.0  0.00  0.00  0.0  0.00  0.0  0.0  0.0  0.0   
2   0.0  0.0  1.81  0.00  0.0  0.00  0.00  0.0  0.00  0.0  0.0  0.0  0.0   
3   0.0  0.0  0.00  1.59  0.0  0.00  0.00  0.0  0.00  0.0  0.0  0.0  0.0   
4   0.0  0.0  0.00  0.00  2.5  0.00  0.00  0.0  0.00  0.0  0.0  0.0  0.0   
5   0.0  0.0  0.00  0.00  0.0  1.81  0.00  0.0  0.00  0.0  0.0  0.0  0.0   
6   0.0  0.0  0.00  0.00  0.0  0.00  1.81  0.0  0.00  0.0  0.0  0.0  0.0   
7   0.0  0.0  0.00  0.00  0.0  0.00  0.00  2.1  0.00  0.0  0.0  0.0  0.0   
8   0.0  0.0  0.00  0.00  0.0  0.00  0.00  0.0  1.81  0.0  0.0  0.0  0.0   
9   0.0  0.0  0.00  0.00  0.0  0.00  0.00  0.0  0.00  2.5  0.0  0.0  0.0   
10  0.0  0.0  0.00  0.00  0.0  0.00  0.00  0.0  0.00  0.0  2.1  0.0  0.0   
11  0.0  0.0  0.00  0.00  0.0  0.00  0.00  0.0  0.00  0.0  0.0  2.5  0.0   
12  0.0  0.0  0.00  0.00  0.0  0.00  0.00  0.0  0.00  0.0  0.0  0.0  2.5   
13  0.0  0.0  0.00  0.00  0.0  0.00  0.00  0.0  0.00  0.0  0.0  0.0  0.0   
14  0.0  0.0  0.00  0.00  0.0  0.00  0.00  0.0  0.00  0.0  0.0  0.0  0.0   
15  0.0  0.0  0.00  0.00  0.0  0.00  0.00  0.0  0.00  0.0  0.0  0.0  0.0   
16  0.0  0.0  0.00  0.00  0.0  0.00  0.00  0.0  0.00  0.0  0.0  0.0  0.0   
17  0.0  0.0  0.00  0.00  0.0  0.00  0.00  0.0  0.00  0.0  0.0  0.0  0.0   
18  0.0  0.0  0.00  0.00  0.0  0.00  0.00  0.0  0.00  0.0  0.0  0.0  0.0   
19  0.0  0.0  0.00  0.00  0.0  0.00  0.00  0.0  0.00  0.0  0.0  0.0  0.0   

      13   14    15   16    17   18   19  
0   0.00  0.0  0.00  0.0  0.00  0.0  0.0  
1   0.00  0.0  0.00  0.0  0.00  0.0  0.0  
2   0.00  0.0  0.00  0.0  0.00  0.0  0.0  
3   0.00  0.0  0.00  0.0  0.00  0.0  0.0  
4   0.00  0.0  0.00  0.0  0.00  0.0  0.0  
5   0.00  0.0  0.00  0.0  0.00  0.0  0.0  
6   0.00  0.0  0.00  0.0  0.00  0.0  0.0  
7   0.00  0.0  0.00  0.0  0.00  0.0  0.0  
8   0.00  0.0  0.00  0.0  0.00  0.0  0.0  
9   0.00  0.0  0.00  0.0  0.00  0.0  0.0  
10  0.00  0.0  0.00  0.0  0.00  0.0  0.0  
11  0.00  0.0  0.00  0.0  0.00  0.0  0.0  
12  0.00  0.0  0.00  0.0  0.00  0.0  0.0  
13  1.81  0.0  0.00  0.0  0.00  0.0  0.0  
14  0.00  2.1  0.00  0.0  0.00  0.0  0.0  
15  0.00  0.0  1.81  0.0  0.00  0.0  0.0  
16  0.00  0.0  0.00  2.1  0.00  0.0  0.0  
17  0.00  0.0  0.00  0.0  1.81  0.0  0.0  
18  0.00  0.0  0.00  0.0  0.00  2.5  0.0  
19  0.00  0.0  0.00  0.0  0.00  0.0  2.5

In [88]:
# compute tfidf feature matrix
tf = np.array(bow_features, dtype='float64')
tfidf = tf * idf
# view raw tfidf feature matrix
pd.DataFrame(np.round(tfidf, 2), columns=feature_names)

bacon  beans  beautiful  blue  breakfast  brown   dog  eggs   fox  green  \
0    0.0    0.0       1.81  1.59        0.0   0.00  0.00   0.0  0.00    0.0   
1    0.0    0.0       1.81  1.59        0.0   0.00  0.00   0.0  0.00    0.0   
2    0.0    0.0       0.00  0.00        0.0   1.81  1.81   0.0  1.81    0.0   
3    2.1    2.5       0.00  0.00        2.5   0.00  0.00   2.1  0.00    0.0   
4    2.1    0.0       0.00  0.00        0.0   0.00  0.00   2.1  0.00    2.5   
5    0.0    0.0       0.00  1.59        0.0   1.81  1.81   0.0  1.81    0.0   
6    0.0    0.0       1.81  1.59        0.0   0.00  0.00   0.0  0.00    0.0   
7    0.0    0.0       0.00  0.00        0.0   1.81  1.81   0.0  1.81    0.0   

   ham  jumps  kings  lazy  love  quick  sausages   sky  toast  today  
0  0.0    0.0    0.0  0.00   0.0   0.00       0.0  1.81    0.0    0.0  
1  0.0    0.0    0.0  0.00   2.1   0.00       0.0  1.81    0.0    0.0  
2  0.0    2.5    0.0  1.81   0.0   1.81       0.0  0.00    0.0    0.0  
3  2.1    0.0    2.5  0.00   0.0   0.00       2.1  0.00    2.5    0.0  
4  2.1    0.0    0.0  0.00   2.1   0.00       2.1  0.00    0.0    0.0  
5  0.0    0.0    0.0  1.81   0.0   1.81       0.0  0.00    0.0    0.0  
6  0.0    0.0    0.0  0.00   0.0   0.00       0.0  3.62    0.0    2.5  
7  0.0    0.0    0.0  1.81   0.0   1.81       0.0  0.00    0.0    0.0

In [89]:
from numpy.linalg import norm
# compute L2 norms 
norms = norm(tfidf, axis=1)

# print norms for each document
print (np.round(norms, 3))

[ 3.013  3.672  4.761  6.534  5.319  4.35   5.019  4.049]


In [90]:
# compute normalized tfidf
norm_tfidf = tfidf / norms[:, None]

# show final tfidf feature matrix
pd.DataFrame(np.round(norm_tfidf, 2), columns=feature_names)

bacon  beans  beautiful  blue  breakfast  brown   dog  eggs   fox  green  \
0   0.00   0.00       0.60  0.53       0.00   0.00  0.00  0.00  0.00   0.00   
1   0.00   0.00       0.49  0.43       0.00   0.00  0.00  0.00  0.00   0.00   
2   0.00   0.00       0.00  0.00       0.00   0.38  0.38  0.00  0.38   0.00   
3   0.32   0.38       0.00  0.00       0.38   0.00  0.00  0.32  0.00   0.00   
4   0.39   0.00       0.00  0.00       0.00   0.00  0.00  0.39  0.00   0.47   
5   0.00   0.00       0.00  0.37       0.00   0.42  0.42  0.00  0.42   0.00   
6   0.00   0.00       0.36  0.32       0.00   0.00  0.00  0.00  0.00   0.00   
7   0.00   0.00       0.00  0.00       0.00   0.45  0.45  0.00  0.45   0.00   

    ham  jumps  kings  lazy  love  quick  sausages   sky  toast  today  
0  0.00   0.00   0.00  0.00  0.00   0.00      0.00  0.60   0.00    0.0  
1  0.00   0.00   0.00  0.00  0.57   0.00      0.00  0.49   0.00    0.0  
2  0.00   0.53   0.00  0.38  0.00   0.38      0.00  0.00   0.00    0.0  
3  0.32   0.00   0.38  0.00  0.00   0.00      0.32  0.00   0.38    0.0  
4  0.39   0.00   0.00  0.00  0.39   0.00      0.39  0.00   0.00    0.0  
5  0.00   0.00   0.00  0.42  0.00   0.42      0.00  0.00   0.00    0.0  
6  0.00   0.00   0.00  0.00  0.00   0.00      0.00  0.72   0.00    0.5  
7  0.00   0.00   0.00  0.45  0.00   0.45      0.00  0.00   0.00    0.0

In [97]:
new_doc = 'the sky is green today'

pd.DataFrame(np.round(tv.transform([new_doc]).toarray(), 2), 
             columns=tv.get_feature_names())

bacon  beans  beautiful  blue  breakfast  brown  dog  eggs  fox  green  \
0    0.0    0.0        0.0   0.0        0.0    0.0  0.0   0.0  0.0   0.63   

   ham  jumps  kings  lazy  love  quick  sausages   sky  toast  today  
0  0.0    0.0    0.0   0.0   0.0    0.0       0.0  0.46    0.0   0.63

# Document Similarity

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(tv_matrix)
similarity_df = pd.DataFrame(similarity_matrix)
similarity_df

0         1         2         3         4         5         6  \
0  1.000000  0.820599  0.000000  0.000000  0.000000  0.192353  0.817246   
1  0.820599  1.000000  0.000000  0.000000  0.225489  0.157845  0.670631   
2  0.000000  0.000000  1.000000  0.000000  0.000000  0.791821  0.000000   
3  0.000000  0.000000  0.000000  1.000000  0.506866  0.000000  0.000000   
4  0.000000  0.225489  0.000000  0.506866  1.000000  0.000000  0.000000   
5  0.192353  0.157845  0.791821  0.000000  0.000000  1.000000  0.115488   
6  0.817246  0.670631  0.000000  0.000000  0.000000  0.115488  1.000000   
7  0.000000  0.000000  0.850516  0.000000  0.000000  0.930989  0.000000   

          7  
0  0.000000  
1  0.000000  
2  0.850516  
3  0.000000  
4  0.000000  
5  0.930989  
6  0.000000  
7  1.000000

## Clustering documents using similarity features

In [14]:
from scipy.cluster.hierarchy import dendrogram, linkage

Z = linkage(similarity_matrix, 'ward')
pd.DataFrame(Z, columns=['Document\Cluster 1', 'Document\Cluster 2', 
                         'Distance', 'Cluster Size'], dtype='object')

Document\Cluster 1 Document\Cluster 2  Distance Cluster Size
0                  2                  7  0.253098            2
1                  0                  6  0.308539            2
2                  5                  8  0.386952            3
3                  1                  9  0.489845            3
4                  3                  4  0.732945            2
5                 11                 12   2.69565            5
6                 10                 13   3.45108            8

In [15]:
plt.figure(figsize=(8, 3))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Data point')
plt.ylabel('Distance')
dendrogram(Z)
plt.axhline(y=1.0, c='k', ls='--', lw=0.5)

In [16]:
from scipy.cluster.hierarchy import fcluster
max_dist = 1.0

cluster_labels = fcluster(Z, max_dist, criterion='distance')
cluster_labels = pd.DataFrame(cluster_labels, columns=['ClusterLabel'])
pd.concat([corpus_df, cluster_labels], axis=1)

Document  \
0                                      The sky is blue and beautiful.   
1                                   Love this blue and beautiful sky!   
2                        The quick brown fox jumps over the lazy dog.   
3  A king's breakfast has sausages, ham, bacon, eggs, toast and beans   
4                         I love green eggs, ham, sausages and bacon!   
5                    The brown fox is quick and the blue dog is lazy!   
6            The sky is very blue and the sky is very beautiful today   
7                         The dog is lazy but the brown fox is quick!   

  Category  ClusterLabel  
0  weather             2  
1  weather             2  
2  animals             1  
3     food             3  
4     food             3  
5  animals             1  
6  weather             2  
7  animals             1

# Topic Models

In [17]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_topics=3, max_iter=10000, random_state=0)
dt_matrix = lda.fit_transform(cv_matrix)
features = pd.DataFrame(dt_matrix, columns=['T1', 'T2', 'T3'])
features

C:\Program Files\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


T1        T2        T3
0  0.832191  0.083480  0.084329
1  0.863554  0.069100  0.067346
2  0.047794  0.047776  0.904430
3  0.037243  0.925559  0.037198
4  0.049121  0.903076  0.047802
5  0.054901  0.047778  0.897321
6  0.888287  0.055697  0.056016
7  0.055704  0.055689  0.888607

## Show topics and their weights

In [18]:
tt_matrix = lda.components_
for topic_weights in tt_matrix:
    topic = [(token, weight) for token, weight in zip(vocab, topic_weights)]
    topic = sorted(topic, key=lambda x: -x[1])
    topic = [item for item in topic if item[1] > 0.6]
    print(topic)
    print()


[('sky', 4.3324395825632624), ('blue', 3.373753174831771), ('beautiful', 3.3323652405224857), ('today', 1.3325579841038182), ('love', 1.3304224288080069)]

[('bacon', 2.332695948479998), ('eggs', 2.332695948479998), ('ham', 2.332695948479998), ('sausages', 2.332695948479998), ('love', 1.335454457601996), ('beans', 1.332773525378464), ('breakfast', 1.332773525378464), ('kings', 1.332773525378464), ('toast', 1.332773525378464), ('green', 1.3325433207547732)]

[('brown', 3.3323474595768783), ('dog', 3.3323474595768783), ('fox', 3.3323474595768783), ('lazy', 3.3323474595768783), ('quick', 3.3323474595768783), ('jumps', 1.3324193736202712), ('blue', 1.2919635624485213)]



## Clustering documents using topic model features

In [19]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=3, random_state=0)
km.fit_transform(features)
cluster_labels = km.labels_
cluster_labels = pd.DataFrame(cluster_labels, columns=['ClusterLabel'])
pd.concat([corpus_df, cluster_labels], axis=1)

Document  \
0                                      The sky is blue and beautiful.   
1                                   Love this blue and beautiful sky!   
2                        The quick brown fox jumps over the lazy dog.   
3  A king's breakfast has sausages, ham, bacon, eggs, toast and beans   
4                         I love green eggs, ham, sausages and bacon!   
5                    The brown fox is quick and the blue dog is lazy!   
6            The sky is very blue and the sky is very beautiful today   
7                         The dog is lazy but the brown fox is quick!   

  Category  ClusterLabel  
0  weather             2  
1  weather             2  
2  animals             1  
3     food             0  
4     food             0  
5  animals             1  
6  weather             2  
7  animals             1